# Демонстрация автоматического расчета важностей

In [1]:
# расширяем поле, убираем предупреждения
from IPython.core.display import display, HTML
display(HTML('<style>.container {width:95% !important;}</style>'))

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Стандартные библиотеки для анализа
import pandas as pd
pd.options.display.float_format = '{:20,.2f}'.format

from sklearn.model_selection import train_test_split, StratifiedKFold

from sklearn.ensemble import RandomForestClassifier

import matplotlib.pylab as pl

In [3]:
import sys
sys.path.append('../')

# Наша разработанная библиотека
from autobinary import SentColumns, CatBoostEncoder, AutoTrees, base_pipe

In [4]:
import sklearn as sk
import xgboost as xgb
import matplotlib as mplot
import autobinary as ab

print('sklearn: ', sk.__version__)
#print('catboost :', catb.__version__)
print('matplotlib: ', mplot.__version__)
print('autobinary: ', ab.__version__)

sklearn:  0.24.2
matplotlib:  3.1.0
autobinary:  1.0.4


In [5]:
from sklearn.datasets import make_classification

X, y = make_classification(
    n_samples=90000, 
    n_features=25, 
    n_informative=16, 
    n_redundant=7, 
    n_classes=4, 
    random_state=42)

sample = pd.DataFrame(X)
sample.columns = [f'feature_{i}' for i in range(len(sample.columns))]

sample['target'] = y

print('Размерность данных:', sample.shape)
sample.head(2)

Размерность данных: (90000, 26)


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,target
0,-2.11,-1.09,-0.88,-4.76,-1.09,2.91,6.00,1.25,-1.34,-2.14,...,-0.88,-0.76,-0.26,-0.77,0.78,-6.97,1.89,1.78,-9.54,3
1,0.73,-0.08,-4.22,3.68,-0.07,0.57,0.30,-0.13,-3.58,-1.54,...,8.49,3.08,0.60,0.31,2.89,-2.56,1.71,-1.23,-5.68,2


In [6]:
num_columns = sample.drop(columns='target').columns.tolist()

cat_columns = [
]

target = ['target']

In [7]:
X_train, X_val, y_train, y_val = train_test_split(
    sample[num_columns+cat_columns],
    sample[target],
    test_size=0.3,
    stratify=sample[target],
    random_state=42
)

In [8]:
prep_pipe = base_pipe(
    num_columns=num_columns, 
    cat_columns=cat_columns)

In [9]:
params = {
    'criterion':'gini',
    'max_depth':5,
    'random_state':42,
    'n_estimators':50}

fit_params = {}

# создаем экземпляр класса XGBClassifier
rf_model = RandomForestClassifier(**params)

# задаем стратегию проверки
strat = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42)

In [10]:
# shift + tab -> вызов опивания класса, в котором прописаны метрики и алгоритмы для решения задач

model = AutoTrees(
    main_estimator=rf_model, 
    main_fit_params=fit_params,
    main_prep_pipe = prep_pipe,
    main_features=num_columns+cat_columns,
    
    X_train=X_train,
    y_train=y_train,
    
    main_metric='f1_macro',
    model_type = 'randomforest')

In [11]:
model.model_fit_cv(strat=strat)

0it [00:00, ?it/s]

==================== Обучение 1 фолда! ====================


1it [00:04,  4.58s/it]

f1_macro на обучающей выборке: 0.707
f1_macro на проверочной выборке: 0.690

********************** 1 фолд обучен! ******************************


==================== Обучение 2 фолда! ====================


2it [00:08,  4.48s/it]

...

f1_macro на обучающей выборке: 0.703
f1_macro на проверочной выборке: 0.696

********************** 3 фолд обучен! ******************************


==================== Обучение 4 фолда! ====================


4it [00:18,  4.53s/it]

f1_macro на обучающей выборке: 0.706
f1_macro на проверочной выборке: 0.693

********************** 4 фолд обучен! ******************************


==================== Обучение 5 фолда! ====================


5it [00:22,  4.53s/it]

f1_macro на обучающей выборке: 0.704
f1_macro на проверочной выборке: 0.700

********************** 5 фолд обучен! ******************************




In [12]:
model.get_mean_cv_scores()

0.6956037639755831

In [13]:
model.get_extra_scores()

,names,fold_1,fold_2,fold_3,fold_4,fold_5
0,f1_macro_train,0.71,0.70,0.70,0.71,0.70
1,f1_macro_valid,0.69,0.70,0.70,0.69,0.70
2,precision_macro_train,0.71,0.71,0.71,0.71,0.71
3,precision_macro_valid,0.69,0.70,0.70,0.70,0.70
4,recall_macro_train,0.71,0.71,0.70,0.71,0.71
5,recall_macro_valid,0.69,0.70,0.70,0.70,0.70


In [14]:
fi = model.get_fi()
fi

,index,importance 0,importance 1,importance 2,importance 3,importance 4,mean_importance
0,feature_5,0.17,0.17,0.17,0.17,0.17,0.17
1,feature_24,0.11,0.11,0.11,0.10,0.11,0.11
2,feature_17,0.08,0.08,0.09,0.08,0.08,0.08
3,feature_0,0.08,0.08,0.08,0.08,0.08,0.08
4,feature_13,0.07,0.07,0.07,0.07,0.07,0.07
5,feature_15,0.07,0.06,0.06,0.06,0.06,0.06
6,feature_1,0.05,0.05,0.05,0.05,0.05,0.05
7,feature_8,0.04,0.05,0.04,0.04,0.05,0.05
8,feature_19,0.04,0.04,0.04,0.04,0.04,0.04
9,feature_9,0.03,0.03,0.03,0.04,0.03,0.03
